In [105]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize

In [106]:
#Kolonner i datasettet som er av typen Json, og må flates ut.
json_columns = [
    'device', 'geoNetwork', 'totals', 'trafficSource'
]
train_path ='../oblig_4/data/train.csv'

In [107]:
load_train = pd.read_csv(train_path, converters={column: json.loads for column in json_columns},
                    dtype={'fullVisitorId':'str'})

In [108]:
# Kilde: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html?highlight=json_normalize
# Kilde: https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields
# Kilde: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html?highlight=merge#pandas.DataFrame.merge
def flatten_json(df, json_columns):
    
    for column in json_columns:
        # Normaliserer json-data til en flat tabell.
        column_as_ch = json_normalize(df[column])
        
        # Navngir de nye subkolonnene som kommer av normaliseringen.
        column_as_ch.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_ch.columns]
        
        # Dropper den gamle kolonnen, fletter de nye kolonnene inn i chunken.
        df = df.drop(column, axis=1).merge(column_as_ch, right_index=True, left_index=True)
    return df

In [109]:
train = flatten_json(load_train, json_columns) #Henter datasett og transformerer json-kolonner.

In [110]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    903653 non-null  object
 1   date                                               903653 non-null  int64 
 2   fullVisitorId                                      903653 non-null  object
 3   sessionId                                          903653 non-null  object
 4   socialEngagementType                               903653 non-null  object
 5   visitId                                            903653 non-null  int64 
 6   visitNumber                                        903653 non-null  int64 
 7   visitStartTime                                     903653 non-null  int64 
 8   device_browser                                     903653 non-null  object
 9   devi

In [111]:
train.isnull().sum()

channelGrouping                                           0
date                                                      0
fullVisitorId                                             0
sessionId                                                 0
socialEngagementType                                      0
visitId                                                   0
visitNumber                                               0
visitStartTime                                            0
device_browser                                            0
device_browserVersion                                     0
device_browserSize                                        0
device_operatingSystem                                    0
device_operatingSystemVersion                             0
device_isMobile                                           0
device_mobileDeviceBranding                               0
device_mobileDeviceModel                                  0
device_mobileInputSelector              

In [112]:
train['totals_transactionRevenue'].unique()

array([nan, '37860000', '306670000', ..., '335260000', '30750000',
       '69390000'], dtype=object)

In [113]:
pd.set_option('display.max_columns', None)
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation,totals_visits,totals_hits,totals_pageviews,totals_bounces,totals_newVisits,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_isTrueDirect,trafficSource_referralPath,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adContent,trafficSource_campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Asia,Western Asia,Turkey,Izmir,(not set),Izmir,not available in demo dataset,ttnet.com.tr,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Southern Europe,Spain,Community of Madrid,(not set),Madrid,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,Linux,not available in demo dataset,False,not ava

In [114]:
train = train.drop([
    'trafficSource_adwordsClickInfo.page', 
    'trafficSource_adwordsClickInfo.slot', 
    'trafficSource_adwordsClickInfo.gclId',
    'trafficSource_adwordsClickInfo.adNetworkType',
    'trafficSource_adwordsClickInfo.isVideoAd',
    'trafficSource_keyword',
    'trafficSource_campaign',
    'trafficSource_adwordsClickInfo.criteriaParameters',
    'trafficSource_adContent',
    'trafficSource_campaignCode',
    'trafficSource_source',
    'device_browserVersion',
    'device_browserSize',
    'device_operatingSystemVersion',
    'device_mobileDeviceBranding',
    'device_mobileDeviceModel',
    'device_mobileInputSelector',
    'device_mobileDeviceInfo',
    'device_mobileDeviceMarketingName',
    'device_flashVersion',
    'device_language',
    'device_screenColors',
    'device_screenResolution',
    'geoNetwork_latitude',
    'geoNetwork_longitude',
    'geoNetwork_metro',
    'geoNetwork_cityId',
    'geoNetwork_city',
    'geoNetwork_region',
    'geoNetwork_networkLocation',
    'geoNetwork_networkDomain',
    'geoNetwork_subContinent',
    'date',
    'visitStartTime',
    'sessionId',
    'trafficSource_medium',
    'trafficSource_referralPath'
], axis = 1)

In [115]:
# Etter fjerning
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   channelGrouping             903653 non-null  object
 1   fullVisitorId               903653 non-null  object
 2   socialEngagementType        903653 non-null  object
 3   visitId                     903653 non-null  int64 
 4   visitNumber                 903653 non-null  int64 
 5   device_browser              903653 non-null  object
 6   device_operatingSystem      903653 non-null  object
 7   device_isMobile             903653 non-null  bool  
 8   device_deviceCategory       903653 non-null  object
 9   geoNetwork_continent        903653 non-null  object
 10  geoNetwork_country          903653 non-null  object
 11  totals_visits               903653 non-null  object
 12  totals_hits                 903653 non-null  object
 13  totals_pageviews            9

In [116]:
train['fullVisitorId'].astype('float')

0         1.131660e+18
1         3.773060e+17
2         3.895546e+18
3         4.763447e+18
4         2.729444e+16
              ...     
903648    5.123779e+18
903649    7.231729e+18
903650    5.744577e+18
903651    2.709355e+18
903652    8.149002e+17
Name: fullVisitorId, Length: 903653, dtype: float64

In [117]:
from pycaret.regression import *

In [118]:
high_cardinality_list = [
    'geoNetwork_country',
    'geoNetwork_continent',
    'totals_pageviews',
    'totals_hits'
]

In [119]:
#Initialiser pycaret
exp_reg = setup(
    train, 
    target = 'totals_transactionRevenue',
    high_cardinality_features = high_cardinality_list,
    normalize = True,
    feature_selection = True,
    ignore_features = ['fullVisitorId']
)

 
Setup Succesfully Completed.


,Description,Value
0,session_id,6995
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(903653, 18)"
4,Missing Values,True
5,Numeric Features,3
6,Categorical Features,14
7,Ordinal Features,False
8,High Cardinality Features,True
9,High Cardinality Method,frequency


In [120]:
exp_reg[0].columns

Index(['device_operatingSystem_Android', 'device_deviceCategory_tablet',
       'channelGrouping_(Other)', 'visitId', 'device_browser_Chrome',
       'device_operatingSystem_iOS', 'geoNetwork_country',
       'device_browser_Opera', 'totals_visits_1',
       'channelGrouping_Paid Search', 'totals_bounces_not_available',
       'device_operatingSystem_Windows Phone',
       'trafficSource_isTrueDirect_not_available',
       'socialEngagementType_Not Socially Engaged',
       'channelGrouping_Organic Search', 'device_operatingSystem_Chrome OS',
       'channelGrouping_Referral', 'device_isMobile_True',
       'device_browser_Safari (in-app)', 'trafficSource_isTrueDirect_True',
       'visitNumber', 'geoNetwork_continent', 'channelGrouping_Social',
       'channelGrouping_Affiliates', 'totals_pageviews',
       'device_browser_Internet Explorer', 'device_operatingSystem_Linux',
       'device_browser_Firefox', 'totals_newVisits_1',
       'device_deviceCategory_mobile', 'totals_newVisits_

In [121]:
comparing_models = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Elastic Net,1243.8722,2126964.8081,1458.0717,0.0028,0.9597,5.7213,0.0089
1,Bayesian Ridge,1243.4359,2126995.1930,1458.0798,0.0028,0.9597,5.7252,0.0220
2,Orthogonal Matching Pursuit,1242.9104,2129442.9167,1458.9035,0.0017,0.9600,5.7251,0.0089
3,Lasso Regression,1243.0480,2130014.7033,1459.1187,0.0014,0.9602,5.7533,0.0211
4,AdaBoost Regressor,1245.6798,2131106.6784,1459.4597,0.0010,0.9630,5.8642,0.2153
5,Support Vector Machine,1245.1988,2131933.4856,1459.7871,0.0005,0.9560,5.6768,3.8624
6,TheilSen Regressor,1242.4410,2131953.8207,1459.7923,0.0005,0.9581,5.6857,3.6139
7,Ridge Regression,1243.8476,2133850.0799,1460.4326,-0.0004,0.9605,5.7595,0.0079
8,Lasso Least Angle Regression,1246.9074,2134920.6211,1460.8008,-0.0009,0.9602,5.7640,0.0092
9,Gradient Boosting Regressor,1241.1434,2137328.6736,1461.5907,-0.0020,0.9593,5.7582,0.7991


In [122]:
model_pareg = create_model('par')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1253.1317,2186956.5167,1478.8362,-0.0069,0.9964,11.4793
1,1225.1028,2114589.4057,1454.1628,-0.0287,0.9301,2.3924
2,1221.1003,2057688.0160,1434.4644,-0.0136,0.8902,4.6246
3,1290.2126,2261667.9726,1503.8843,-0.0181,0.9813,5.5779
4,1228.1887,2146525.7535,1465.1026,-0.0138,0.9341,2.0982
5,1288.4035,2251949.2485,1500.6496,-0.0016,1.0132,9.7784
6,1267.3782,2225886.8036,1491.9406,-0.0075,0.9835,5.5799
7,1233.6653,2121425.2572,1456.5113,-0.0195,0.9107,2.1136
8,1263.0942,2215034.2533,1488.2991,0.0060,0.9676,5.5131
9,1176.6558,1974450.6876,1405.1515,-0.0036,0.9473,6.3944


In [123]:
tuned_pareg = tune_model(model_pareg)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1251.1420,2181720.9036,1477.0650,-0.0045,0.9924,11.4042
1,1224.0756,2113281.8846,1453.7131,-0.0280,0.9286,2.4003
2,1211.6691,2036772.7314,1427.1555,-0.0033,0.8762,4.3463
3,1286.8542,2249633.6041,1499.8779,-0.0127,0.9752,5.7718
4,1234.1095,2179117.9663,1476.1836,-0.0292,0.9170,1.9596
5,1293.0823,2278962.1303,1509.6232,-0.0136,1.0033,9.5206
6,1275.9571,2252120.7118,1500.7067,-0.0193,0.9871,5.4777
7,1229.1175,2108109.4892,1451.9330,-0.0131,0.9084,2.0869
8,1260.5993,2185331.6765,1478.2867,0.0193,0.9785,5.8005
9,1176.2703,1974443.0857,1405.1488,-0.0036,0.9501,6.1977


In [124]:
model_svm = create_model('svm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1252.2426,2168725.1332,1472.6592,0.0015,1.0058,11.4057
1,1217.2217,2058085.3454,1434.6029,-0.0012,0.9339,2.4510
2,1214.8894,2026997.0242,1423.7265,0.0015,0.8844,4.6317
3,1283.0083,2218732.1941,1489.5409,0.0012,0.9783,6.3340
4,1231.3669,2114872.3346,1454.2601,0.0011,0.9283,2.0478
5,1293.3040,2243492.1135,1497.8291,0.0022,1.0157,9.8113
6,1273.6827,2208028.5082,1485.9436,0.0006,0.9797,5.5105
7,1230.2688,2082295.4975,1443.0161,-0.0007,0.9064,2.0831
8,1277.9423,2226141.6590,1492.0260,0.0010,0.9813,6.2604
9,1178.0615,1971965.0465,1404.2667,-0.0023,0.9457,6.2323


In [125]:
tuned_svm = tune_model(model_svm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1248.4380,2162438.6506,1470.5233,0.0043,1.0050,11.6433
1,1211.9092,2049973.1798,1431.7727,0.0028,0.9336,2.4696
2,1210.6482,2019836.7262,1421.2096,0.0051,0.8834,4.6491
3,1281.1779,2219194.6552,1489.6962,0.0010,0.9788,6.1921
4,1223.3397,2107974.8684,1451.8867,0.0044,0.9277,2.0526
5,1288.5866,2232386.6911,1494.1174,0.0071,1.0158,9.8796
6,1267.8241,2206204.8090,1485.3299,0.0014,0.9790,5.4336
7,1227.8916,2076490.5228,1441.0033,0.0021,0.9065,2.0893
8,1268.2878,2204362.7773,1484.7097,0.0107,0.9785,6.1126
9,1173.3754,1964803.0362,1401.7143,0.0013,0.9451,6.1506


In [126]:
create_gbr = create_model('gbr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1247.7521,2191013.9980,1480.2074,-0.0088,1.0116,11.8077
1,1212.4943,2060674.5504,1435.5050,-0.0024,0.9365,2.4752
2,1204.3902,2005017.2980,1415.9863,0.0124,0.8819,4.1193
3,1286.8713,2234362.5496,1494.7784,-0.0058,0.9838,6.4818
4,1218.1026,2101676.2193,1449.7159,0.0073,0.9305,2.0777
5,1287.5812,2253932.2702,1501.3102,-0.0025,1.0189,9.9967
6,1270.6981,2222152.9777,1490.6888,-0.0058,0.9837,6.0733
7,1238.5363,2117936.2707,1455.3131,-0.0178,0.9157,2.1559
8,1267.0458,2214719.6271,1488.1934,0.0061,0.9807,6.0533
9,1177.9624,1971800.9745,1404.2083,-0.0022,0.9497,6.3409


In [130]:
stacker = stack_models(estimator_list = [tuned_pareg, tuned_svm], meta_model = create_gbr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1243.8305,2162234.9509,1470.4540,0.0044,1.0102,11.8625
1,1215.3820,2059748.6455,1435.1824,-0.0020,0.9365,2.4955
2,1208.3416,2008978.2697,1417.3843,0.0104,0.8826,4.3264
3,1288.3840,2239011.7759,1496.3328,-0.0079,0.9851,6.2755
4,1221.0352,2117309.4827,1455.0978,-0.0000,0.9323,2.0974
5,1295.7347,2269807.6792,1506.5881,-0.0095,1.0225,9.9997
6,1276.8632,2235157.9754,1495.0445,-0.0117,0.9823,5.8478
7,1228.8148,2084136.5082,1443.6539,-0.0016,0.9121,2.1453
8,1274.7713,2265849.5387,1505.2739,-0.0169,0.9832,6.1412
9,1191.6506,2025298.1130,1423.1297,-0.0294,0.9532,6.3004
